In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish


In [7]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

from src.ad_hmm import sku_predict
from src.ad_stl_model import stl_model
from src.utils import *

In [8]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Input data

In [4]:
train_test = pd.read_csv("../../data/train_validation_marker.csv")
train_test["SALES_DATE"] = pd.to_datetime(train_test["SALES_DATE"])
print (train_test.shape)

train_test["if_promo"] = (~train_test["PROMO_PRICE"].isin(["?"])).astype("int")
train_test["if_competitor"] = (~train_test["COMPETITOR_PRICE"].isin(["?"])).astype("int")

train_test["min_sales_price"] = pd.to_numeric(np.where(train_test.if_promo>0, train_test.PROMO_PRICE,\
                                                       train_test.RETAIL_PRICE))

train_test["comp_price_clean"] = pd.to_numeric(np.where(train_test.if_competitor>0,train_test.COMPETITOR_PRICE, np.nan))
train_test["price_delta"] = np.where(train_test.if_competitor>0, train_test.min_sales_price - train_test.comp_price_clean,\
                                   -train_test.min_sales_price)

(846632, 15)


### Pivot data

In [10]:
train_test_pivot = train_test.pivot(index='SALES_DATE', columns='Encoded_SKU_ID',\
                   values=['DAILY_UNITS',"if_promo","if_competitor","min_sales_price","price_delta",\
                          "validation","validation_clean"])
train_test_pivot.columns = train_test_pivot.columns.swaplevel(0, 1)

### Test code for 1 sku

In [11]:
sku_id = 50
train_test_sku = train_test_pivot.loc[:,(sku_id,)].copy()
train_test_sku["ad_time"] = list(range(1,train_test_sku.shape[0]+1))

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/4124826544.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  train_test_sku = train_test_pivot.loc[:,(sku_id,)].copy()


In [14]:
exog_cols = ["if_promo","if_competitor","min_sales_price","price_delta"]
stl_model1 = stl_model(sku_id = sku_id,\
                      train_test_sku = train_test_sku,\
                      n_lags_y = 30, \
                      n_lags_exog = 14, \
                      pred_cols = exog_cols)

In [53]:
m_dict, y_dict = stl_model1.fit_models();
min_models, y_pred = stl_model1.get_result();

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish/../../src/ad_stl_model.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku["STL_resid"] = res.resid
/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish/../../src/ad_stl_model.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sku["STL_trend"] = res.trend
/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish/../../src/ad_stl_model.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

(50, 'STL_resid', 'auto_reg'), Train RMSE = 9.209
(50, 'STL_resid', 'xgb_340'), Train RMSE = 12.52
(50, 'STL_resid', 'xgb_1000'), Train RMSE = 13.28

(50, 'STL_trend', 'auto_reg'), Train RMSE = 0.0005146


/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will c

(50, 'STL_trend', 'xgb_340'), Train RMSE = 11.32
(50, 'STL_trend', 'xgb_1000'), Train RMSE = 11.32

(50, 'STL_seasonal', 'auto_reg'), Train RMSE = 11.35


/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will c

...

(50, 'MSTL_seasonal_7', 'xgb_340'), Train RMSE = 12.09
(50, 'MSTL_seasonal_7', 'xgb_1000'), Train RMSE = 11.76

(50, 'MSTL_seasonal_30', 'auto_reg'), Train RMSE = 1.034


/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will c

(50, 'MSTL_seasonal_30', 'xgb_340'), Train RMSE = 7.873
(50, 'MSTL_seasonal_30', 'xgb_1000'), Train RMSE = 7.772

(50, 'MSTL_seasonal_365', 'auto_reg'), Train RMSE = 9.043


/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ashish1610dhiman/opt/anaconda3/envs/bestbuy/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will c

(50, 'MSTL_seasonal_365', 'xgb_340'), Train RMSE = 13.1
(50, 'MSTL_seasonal_365', 'xgb_1000'), Train RMSE = 13.1



In [57]:
min_models

,component,model_name,model,train_rmse,rnk
0,STL_resid,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,9.20895,1.0
3,STL_trend,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.000515,1.0
6,STL_seasonal,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,11.35479,1.0
9,MSTL_resid,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,6.854014,1.0
12,MSTL_trend,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.000492,1.0
15,MSTL_seasonal_7,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.416672,1.0
18,MSTL_seasonal_30,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,1.034446,1.0
21,MSTL_seasonal_365,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,9.042637,1.0


In [56]:
print (np.mean((y_pred["actual"]-y_pred["STL_prediction"])**2)**0.5,
np.mean((y_pred["actual"]-y_pred["MSTL_prediction"])**2)**0.5)

16.751089295655188 25.065478710017754


### Run for all SKUs

In [31]:
from joblib import Parallel, delayed
from IPython.utils import io
from tqdm import tqdm

In [49]:
def fit_one_sku(train_test_pivot1,sku_id1, n_lags_y1, n_lags_exog1, exog_cols1):
    train_test_sku1 = train_test_pivot1.loc[:,(sku_id1,)].copy()
    train_test_sku1["ad_time"] = list(range(1,train_test_sku1.shape[0]+1))
    stl_model1 = stl_model(sku_id = sku_id1,\
                      train_test_sku = train_test_sku1,\
                      n_lags_y = n_lags_y1, \
                      n_lags_exog = n_lags_exog1, \
                      pred_cols = exog_cols1)
    try:
        with io.capture_output() as captured:
            m_dict, y_dict = stl_model1.fit_models();
            min_models, y_pred = stl_model1.get_result();
        print (f"Successful for {sku_id1}")
        return (min_models, y_pred)
    except:
        print (f"Error for {sku_id1}")

In [50]:
clean_SKUs = train_test[train_test.validation_clean==True]["Encoded_SKU_ID"].unique()

In [58]:
all_preds = Parallel(n_jobs=5)(delayed(fit_one_sku)(train_test_pivot1 = train_test_pivot,\
                                                    sku_id1 = sku_id_i,\
                                                    n_lags_y1 = 30,\
                                                    n_lags_exog1 = 14,\
                                                    exog_cols1 = exog_cols) \
                               for sku_id_i in tqdm(clean_SKUs))


  0%|                                                                                                                                                  | 0/545 [00:00<?, ?it/s]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

  2%|██▍                                             

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

  9%|████████████▏                                                                                                                        | 50/545 [1:32:59<7:20:33, 53.40s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyke

 17%|█████████████████████▉                                                                                                               | 90/545 [1:48:57<3:10:47, 25.16s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 17%|███████████████████████▏                         

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 25%|████████████████████████████████▋                                                                                                   | 135/545 [2:06:34<2:46:18, 24.34s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyke

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 33%|███████████████████████████████████████████▌                                                                                        | 180/545 [2:24:34<2:18:41, 22.80s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyke

...


 64%|████████████████████████████████████████████████████████████████████████████████████▊                                               | 350/545 [3:35:46<1:23:39, 25.74s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 65%|████████████████████████████████████████████████

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 72%|███████████████████████████████████████████████████████████████████████████████████████████████▋                                    | 395/545 [4:21:06<4:01:44, 96.70s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyke

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 81%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                         | 440/545 [4:36:16<38:41, 22.11s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyke


 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                | 480/545 [4:47:49<15:52, 14.66s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 89%|████████████████████████████████████████████████

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████     | 525/545 [5:00:02<05:25, 16.30s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_23586/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyke

In [59]:
len(all_preds)

545

In [61]:
import pickle
with open('../../data/stl_results_v1.pkl', 'wb') as f:
    pickle.dump(all_preds, f)

### RMSE

In [77]:
y_pred_df = pd.DataFrame()
for i in range(len(all_preds)):
    if all_preds[i]!= None:
        y_pred_df = pd.concat([y_pred_df,all_preds[i][1]])

In [80]:
y_pred_df["Encoded_SKU_ID"].nunique()

503

In [213]:
validation = train_test[(train_test.validation==True) & (train_test.validation_clean==True)]
y_act = validation[["Encoded_SKU_ID","SALES_DATE","DAILY_UNITS"]].rename(columns = {"DAILY_UNITS":"actual"})
y_act.head()

,Encoded_SKU_ID,SALES_DATE,actual
100,96,2022-07-27,2
218,187,2022-07-29,0
420,297,2022-07-29,6
652,372,2022-07-27,1
831,412,2022-07-26,4


In [214]:
y_pred_null = pd.read_csv("../../data/null_prediction_validation_result_v0.csv").drop(columns = "Unnamed: 0")
y_pred_null["SALES_DATE"] = pd.to_datetime(y_pred_null["SALES_DATE"])

In [216]:
y_pred_stl = y_pred_df.reset_index()[["SALES_DATE","STL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"STL_prediction":"predicted"})

y_pred_mstl = y_pred_df.reset_index()[["SALES_DATE","MSTL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"MSTL_prediction":"predicted"})

# y_act = y_pred_df.reset_index()[["SALES_DATE","actual","Encoded_SKU_ID"\
#                         ]]
y_act["actual"] = pd.to_numeric(y_act["actual"])

In [217]:
y_pred_null_subset = y_pred_null.loc[y_pred_null.Encoded_SKU_ID.isin(y_pred_stl.Encoded_SKU_ID.unique())]
len(y_pred_null),len(y_pred_null_subset)

(3815, 3521)

In [218]:
rmse(y_act,y_pred_null),rmse(y_act,y_pred_null_subset)

(6.59603916258372, 6.609787652757184)

In [219]:
rmse(y_act,y_pred_stl)

14.416066109391119

In [220]:
rmse(y_act,y_pred_mstl)

13.913722800537084

In [221]:
rmse_sku_stl = rmse_sku(y_act,y_pred_stl)
rmse_sku_mstl = rmse_sku(y_act,y_pred_mstl)
rmse_sku_null = rmse_sku(y_act,y_pred_null)

In [222]:
VERSION = "v2"
hmm_output = pd.read_csv(f"../../data/hmm_result_{VERSION}.csv",index_col=0)

In [223]:
y_act.dtypes

Encoded_SKU_ID             int64
SALES_DATE        datetime64[ns]
actual                     int64
dtype: object

In [224]:
rmse_sku_hmm_dates = pd.DataFrame()
for i,col in enumerate(hmm_output.columns):
    if "pred" in col:
        y_pred_hmm = hmm_output.reset_index(names = "SALES_DATE")[["Encoded_SKU_ID","SALES_DATE",col\
                                                                  ]].rename(columns={col:"predicted"})
        y_pred_hmm["SALES_DATE"] = pd.to_datetime(y_pred_hmm["SALES_DATE"])
        rmse_sku_hmm_i = rmse_sku(y_act,y_pred_hmm)
        if i ==0:
            rmse_sku_hmm_dates = rmse_sku_hmm_i
        else:
            rmse_sku_hmm_dates = rmse_sku_hmm_dates.merge(rmse_sku_hmm_i, left_index = True, right_index = True,\
                                suffixes = ("",f"_{col.split('_')[-1]}"))

In [225]:
other_rmse = rmse_sku_stl.merge(rmse_sku_mstl,on=["Encoded_SKU_ID"],how="outer", suffixes = ("_stl","_mstl")\
                  ).merge(rmse_sku_null,on=["Encoded_SKU_ID"],how="outer", suffixes = ("","_null"))

In [226]:
y_act.Encoded_SKU_ID.nunique()

545

In [227]:
rmse_sku_join = rmse_sku_hmm_dates.merge(other_rmse,left_index=True,right_on=["Encoded_SKU_ID"],how="outer",\
                                        suffixes = ("_15","_null"),)

### Pick Best

In [228]:
rmse_cols = [col for col in rmse_sku_join.columns if "rmse_du" in col]
rmse_cols

['rmse_du_15',
 'rmse_du_30',
 'rmse_du_45',
 'rmse_du_60',
 'rmse_du_90',
 'rmse_du_max',
 'rmse_du_stl',
 'rmse_du_mstl',
 'rmse_du_null']

In [229]:
rmse_sku_join["min_rmse"] = rmse_sku_join[rmse_cols].min(axis=1,skipna=True)

rmse_sku_join["argmin_period"] = np.argmin(rmse_sku_join[rmse_cols].values,axis=1)

rmse_sku_join["min_period"] = rmse_sku_join["argmin_period"].apply(lambda x: rmse_cols[x])

In [230]:
rmse_sku_join["min_rmse"].isna().sum()

0

In [231]:
hmm_output1 = hmm_output.reset_index(names="SALES_DATE")
hmm_output1["SALES_DATE"] = pd.to_datetime(hmm_output1["SALES_DATE"])

In [232]:
all_outputs = hmm_output1.merge(y_pred_df.reset_index(),on=["Encoded_SKU_ID","SALES_DATE"],how="outer"\
                                            ).merge(y_pred_null,on=["Encoded_SKU_ID","SALES_DATE"],how="outer",\
                                                   suffixes = ("_15","_null"))

In [233]:
all_outputs

,SALES_DATE,predicted_15,predicted_30,predicted_45,predicted_60,predicted_90,predicted_max,Encoded_SKU_ID,STL_prediction,MSTL_prediction,actual,predicted_null
0,2022-07-25,1.0,1.0,1.0,0.00,2.00,3.00,1,1.708770,2.087032,0,1
1,2022-07-26,1.0,1.0,2.0,3.00,0.45,2.00,1,1.642804,0.778320,0,1
2,2022-07-27,2.0,0.0,3.0,2.00,2.55,1.00,1,1.464536,1.561164,0,1
3,2022-07-28,0.0,1.0,0.0,0.45,0.45,2.45,1,1.454188,2.314384,2,1
4,2022-07-29,3.0,2.0,1.0,2.00,2.55,2.00,1,1.738205,1.581958,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3810,2022-07-27,NaN,NaN,NaN,NaN,NaN,NaN,566,NaN,NaN,NaN,5
3811,2022-07-28,NaN,NaN,NaN,NaN,NaN,NaN,566,NaN,NaN,NaN,5
3812,2022-07-29,NaN,NaN,NaN,NaN,NaN,NaN,566,NaN,NaN,NaN,5
3813,2022-07-30,NaN,NaN,NaN,NaN,NaN,NaN,566,NaN,NaN,NaN,5


In [234]:
period_col_map= {
    "rmse_du_15":"predicted_15", "rmse_du_30":"predicted_30", "rmse_du_45":"predicted_45",\
    "rmse_du_60":"predicted_60", "rmse_du_90":"predicted_90", "rmse_du_max":"predicted_max",\
    "rmse_du_stl":"STL_prediction","rmse_du_mstl":"MSTL_prediction","rmse_du_null":"predicted_null"
}

In [235]:
y_pred_best = pd.DataFrame()
cnt = 0
for sku_id,best_period in zip(rmse_sku_join.index,rmse_sku_join["min_period"]):
    best_col = period_col_map[best_period]
    mask_sku = all_outputs.Encoded_SKU_ID==sku_id
    y_pred_best_sku = all_outputs.loc[mask_sku][["Encoded_SKU_ID","SALES_DATE",best_col\
                                                                  ]].rename(columns={best_col:"predicted"})
    y_pred_best = pd.concat([y_pred_best,y_pred_best_sku])

In [236]:
y_act.Encoded_SKU_ID.nunique(),y_pred_null.Encoded_SKU_ID.nunique(),rmse_sku_join.index.nunique()

(545, 545, 545)

In [237]:
a,b = rmse(y_act,y_pred_null),rmse(y_act,y_pred_best)
a,b

(6.59603916258372, 1.052294878152332)

In [238]:
(a-b)/a

0.8404656412409565

In [239]:
rmse_sku_join["min_period"].value_counts()

rmse_du_15      225
rmse_du_stl      98
rmse_du_max      44
rmse_du_null     38
rmse_du_30       38
rmse_du_60       31
rmse_du_90       29
rmse_du_45       23
rmse_du_mstl     19
Name: min_period, dtype: int64